In [1]:
import pickle
import torch
# import CNN_LSTM_n
# import TF_data_loader
from torch.utils import data as D
import numpy as np
from tqdm.notebook import tqdm
import gc

First, we make the panset, non-normalized

In [2]:
with open('../../../data/Chipseq_data/seq_breathing_feat.pkl', 'rb') as f:
    seq_feat = pickle.load(f)

In [3]:
feat_map = torch.load('../../../data/Chipseq_data/FeatMap.pt')
feat_map = {v: k for k, v in feat_map.items()}
group_index = torch.load('../../../data/Chipseq_data/group_index.pt')

In [4]:
panset = {}

In [5]:
for partition in seq_feat:
    partition = seq_feat[partition]
    for seq_id in partition:
        panset[seq_id] = partition[seq_id]

In [6]:
TF_map = torch.load('TF_map.pt')

In [7]:
def TF_sorter(panset, TF_map):
    np.random.seed(108)
    data_dict = {}

    progress_bar = tqdm(TF_map, desc='Sorting TFs', unit='TF')

    for TF_name in progress_bar:
        pos_seqs = TF_map[TF_name]['positive']
        neg_seqs = TF_map[TF_name]['negative']

        # Select the appropriate sample size based on the number of negative sequences available
        sample_size = min(len(neg_seqs), len(pos_seqs) * 5)
        neg_indices = np.random.choice(len(neg_seqs), sample_size, replace=False)
        neg_seqs_selected = [neg_seqs[i] for i in neg_indices]

        data_dict[TF_name] = {}
        data_dict[TF_name]['train'] = {}
        data_dict[TF_name]['test'] = {}

        # Populate train and test partitions for positive sequences
        pos_train_count = int(len(pos_seqs) * 0.8)
        pos_train_seqs = pos_seqs[:pos_train_count]
        pos_test_seqs = pos_seqs[pos_train_count:]

        for seq_id in pos_train_seqs:
            data_dict[TF_name]['train'][seq_id] = panset[seq_id]
            data_dict[TF_name]['train'][seq_id]['label'] = np.array([1])

        for seq_id in pos_test_seqs:
            data_dict[TF_name]['test'][seq_id] = panset[seq_id]
            data_dict[TF_name]['test'][seq_id]['label'] = np.array([1])

        # Populate train and test partitions for negative sequences
        neg_train_count = int(len(neg_seqs_selected) * 0.8)
        neg_train_seqs = neg_seqs_selected[:neg_train_count]
        neg_test_seqs = neg_seqs_selected[neg_train_count:]

        for seq_id in neg_train_seqs:
            data_dict[TF_name]['train'][seq_id] = panset[seq_id]
            data_dict[TF_name]['train'][seq_id]['label'] = np.array([0])

        for seq_id in neg_test_seqs:
            data_dict[TF_name]['test'][seq_id] = panset[seq_id]
            data_dict[TF_name]['test'][seq_id]['label'] = np.array([0])
            
        progress_bar.set_description(f'Sorting TFs: {TF_name}')

    return data_dict

In [8]:
data_dict = TF_sorter(panset, TF_map)

Sorting TFs:   0%|          | 0/64 [00:00<?, ?TF/s]

In [9]:
with open('TF_dataset.pickle', 'wb') as f:
    pickle.dump(data_dict, f)

In [2]:
with open('TF_dataset.pickle', 'rb') as f:
    data_dict = pickle.load(f)

In [4]:
for TF_name, partitions in data_dict.items():
    print(f'TF: {TF_name}')
    for partition, seq_ids in partitions.items():
        print(f'Partition: {partition}, Count: {len(seq_ids)}')
    print()

TF: HNF4A
Partition: train, Count: 95164
Partition: test, Count: 23792

TF: JUND
Partition: train, Count: 130051
Partition: test, Count: 32513

TF: NFYB
Partition: train, Count: 110702
Partition: test, Count: 27676

TF: FOXA1
Partition: train, Count: 407889
Partition: test, Count: 101973

TF: SP1
Partition: train, Count: 100180
Partition: test, Count: 25046

TF: EBF1
Partition: train, Count: 206342
Partition: test, Count: 51586

TF: TAF1
Partition: train, Count: 200433
Partition: test, Count: 50109

TF: BCL3
Partition: train, Count: 99724
Partition: test, Count: 24932

TF: CHD2
Partition: train, Count: 140016
Partition: test, Count: 35004

TF: ELF1
Partition: train, Count: 248779
Partition: test, Count: 62195

TF: POLR2A
Partition: train, Count: 681249
Partition: test, Count: 170313

TF: BRCA1
Partition: train, Count: 48086
Partition: test, Count: 12022

TF: ZNF143
Partition: train, Count: 36945
Partition: test, Count: 9237

TF: BHLHE40
Partition: train, Count: 88833
Partition: test, C